<a href="https://colab.research.google.com/github/AuraFrizzati/WebScraping_NSHCS/blob/main/NHSCS_ClinInformatics_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code to web scrape the Clinical Informatics Curriculum from the NHSCS website**

In [1]:
# skip the cells from running
%%script echo skipping
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

skipping


In [2]:
# skip the cells from running
%%script echo skipping
%pip install requests
%pip install html5lib
%pip install bs4
%pip install pandas


skipping


In [3]:
%pip install xlsxwriter
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xlsxwriter
import random
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Custom-made function for filling dictionaries

In [6]:
def fill_dictionary(soup_find_name, soup_find_attr1, soup_find_attr2, tag, 
                    extra_level = False, extra_level_tag = None, extra_level_number = None):
  string = ''
  if (extra_level == False):
    for item in soup.find(soup_find_name, attrs= {soup_find_attr1 : soup_find_attr2}).find_all(tag):
      string = string + item.text + " | "
  if (extra_level == True):
    for item in soup.find(soup_find_name, attrs= {soup_find_attr1 : soup_find_attr2}).find_all(extra_level_tag)[extra_level_number].find_all(tag):
      string = string + item.text + " | "
  
  return(string[:-2])

Extract links for Competencies' web pages

In [22]:
## Select Clinical Informatics curriculum main page
URL = "https://curriculumlibrary.nshcs.org.uk/stp/specialty/SBI1-3-22/"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib') ## specify the HTML content and the parser

## extract Modules links
links = []
for section in soup.find_all('table',attrs= {'class' : 'nhsuk-table-responsive'}):
  for link in section.find_all('a'):
    links.append(link['href'])

links = ['https://curriculumlibrary.nshcs.org.uk' + link for link in links]

In [23]:
links

['https://curriculumlibrary.nshcs.org.uk/stp/module/S-C1',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-C2',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-C3',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-C4',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-R1',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-R2',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-BG-R1',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-CSC-R1',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-S1',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-S2',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-S3',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-S4',
 'https://curriculumlibrary.nshcs.org.uk/stp/module/S-HI-S5']

Create Excel spreadsheet for the curriculum

In [91]:
# Open Excel workbook on Google Cloud
%cd /content/drive/MyDrive/web scraping/output/
writer = pd.ExcelWriter('ClinicalInformatics_curriculum2022.xlsx',engine='xlsxwriter')   
workbook=writer.book


/content/drive/MyDrive/web scraping/output


In [92]:
title_worsheets = []
for link in links:
  r = requests.get(link)
  soup = BeautifulSoup(r.content, 'html5lib') ## specify the HTML content and the parser

  ## [1] Extract Module Details table
  mod_details = {}

  # extract subsection with Module Summary
  for i in range(len(soup.find_all('dt',attrs = {'class':'nhsuk-summary-list__key'}))):
    mod_details[soup.find_all('dt',attrs = {'class':'nhsuk-summary-list__key'})[i].text.strip()] = soup.find_all('dd',attrs = {'class':'nhsuk-summary-list__value'})[i].text.strip()
  # extract subsection with Module Aim
  sub_aim = soup.find('div', attrs= {'class' : 'subsection', 'id':'aim'})
  mod_details['Aim'] = sub_aim.text.strip()
  # add link to online module
  mod_details['Link'] = link
  # transform list into df
  df_mod_details = pd.DataFrame(mod_details.items())
  remove = 'Copy text                        \n                        \n                            Aim of this module\n                            '
  df_mod_details[1] = df_mod_details[1].str.replace(remove, '')

  # [2] Extract Learning Outcomes table
  df_LO = pd.read_html(str(soup.find('div', attrs={'id':'action-wb-learning-outcomes'}).find('table')))[0]

  # [3] extract Assessment table 
  Assessments_dict = {}
  Assessments_dict['Assessments'] = fill_dictionary('div','id','work-based-assessments','p')
  df_Assessments = pd.DataFrame(Assessments_dict.items())
  if soup.find('div', attrs= {'id' : 'work-based-assessments'}).ul is not None:
    Assessments_dict['DOPS'] = fill_dictionary('div','id','work-based-assessments','li', 
                                              extra_level = True, extra_level_tag = 'ul', extra_level_number = 1)
    Assessments_dict['OCEs'] = fill_dictionary('div','id','work-based-assessments','li',
    extra_level = True, extra_level_tag = 'ul', extra_level_number = 2)
    df_Assessments = pd.DataFrame(Assessments_dict.items())
    if soup.find('div', attrs= {'id' : 'action-activities'}) is not None:
      Assessments_dict['Clinical activities'] = fill_dictionary('div','id','action-activities','li')
      df_Assessments = pd.DataFrame(Assessments_dict.items())

  # write Module Summary to Excel
  title_worksheet = mod_details['Module code']+"_summary"
  title_worsheets.append(title_worksheet)
  worksheet=workbook.add_worksheet(title_worksheet)
  writer.sheets[title_worksheet] = worksheet
  df_mod_details.to_excel(writer,sheet_name=title_worksheet, header = False, index = False,
                          startrow=0 , startcol=0)   
  df_LO.to_excel(writer,sheet_name=title_worksheet, index = False,
                startrow=df_mod_details.shape[0]+3, startcol=0) 
  
  df_Assessments.to_excel(writer,sheet_name=title_worksheet, header = False, index = False,
                startrow=df_mod_details.shape[0]+df_LO.shape[0]+6, startcol=0)
  
  # Extract Competencies table
  df_competencies = pd.read_html(str(soup.find('table', attrs={'class':'nhsuk-table-responsive','id':'table-competencies'})))[0] ## read HTML table in pandas
  df_competencies['Learning outcome'] = df_competencies['Learning outcome'].str.replace('Learning outcome', '')
  df_competencies['#'] = mod_details['Module code'] + df_competencies['#'].str.replace('# ', '.') + ":"
  df_competencies['Competency'] = df_competencies['#'] + df_competencies['Competency'].str.replace('Competency', '')
  df_competencies = df_competencies.drop('#', axis=1)
  df_competencies

  Views_URLs = [] 
  for i in range(len(df_competencies.index)):
    U = link.replace("module","competency")+"/"+str((i+1))
    Views_URLs.append(U)
  
  Views_list = []
  for URL in Views_URLs:
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    considerations_list = soup.find('div',attrs={'id':'action-considerations'}).find_all('li')
    considerations_string = ''
    for item in considerations_list:
      considerations_string = considerations_string + item.text + ', '
    considerations_string = considerations_string[:-2]
    Views_list.append(considerations_string)
    time.sleep(random.randint(1,4))
  
  df_competencies['Action'] = Views_list

  # write Module Competencies to Excel
  title_worksheet = mod_details['Module code']+"_competencies"
  title_worsheets.append(title_worksheet)
  worksheet=workbook.add_worksheet(title_worksheet)
  writer.sheets[title_worksheet] = worksheet
  df_competencies.to_excel(writer,sheet_name=title_worksheet, index = False,
                          startrow=0 , startcol=0)   
    
  time.sleep(random.randint(1,4))

In [93]:
## add first worksheet with internal links to the rest of the workbook

# create the internal link worksheet
title_worksheet = "index"
worksheet=workbook.add_worksheet(title_worksheet)

# create the links
i = 0
for item in title_worsheets:
  i = i+1
  worksheet.write_url('A'+str(i),  'internal:'+item+'!A1:B2', string=item)
  #worksheet.write_string('B'+str(i), 'Your text here')

# Move last spreadsheet (index) as first
workbook.worksheets_objs.insert(0, workbook.worksheets_objs.pop())

In [94]:
#Close Excel workbook
writer.close()